In [1]:
import sys
import numpy as np
from numba import jit
sys.path.append("../")
from src.utils import GenOverlapGroup
from src.regularizer import OGL1

In [2]:
p = 7
generator = GenOverlapGroup(p, 3, 5)
starts, ends = generator.get_group()
index = [*range(p)]
for i in range(len(starts)):
    print(index[starts[i]:ends[i]+1])

[0, 1, 2, 3, 4]
[1, 2, 3, 4, 5]
[2, 3, 4, 5, 6]


In [3]:
starts, ends

([0, 1, 2], [4, 5, 6])

In [ ]:
class OGL1:
    def __init__(self, Lambda, dim, starts, ends):
        """
        Lambda: scalar > 0
        starts: a list of numbers speficy the starting index of each group
          ends: a list of numbers speficy the end index of each group

        For example, a overlapping group configuration, the number stands for
        the index of the variable.
        {g0=[0,1,2,3,4],g1=[3,4,5,6,7], g2=[5,6,7,8,9]}
        stars = [0, 3, 5]
        ends  = [4, 7, 9]
        """
        self.p = dim
        self.K = len(starts)
        # a np.array that stores the number of group that each coordinate belongs to
        self.freq = np.zeros((self.p, 1))
        self.group_size = np.zeros(self.K)
        for i in range(len(starts)):
            self.freq[starts[i]:ends[i] + 1] += 1
            self.group_size[i] = ends[i] - starts[i] + 1
        # self.Lambda_group = np.ones(self.K)
        self.Lambda_group = Lambda * np.sqrt(self.group_size)
        self.starts = np.array(starts)
        # since python `start:end` will include `start` and exclude `end`,
        # we add 1 to the `end` so the G_i-th block of X is indexed by X[start:end]
        self.ends = np.array(ends) + 1

    def __str__(self):
        return("Overlapping Group L1")

    def func_ub(self, X, approx=1):
        if approx == 1:
            return _fub1_jit(X, self.K, self.starts, self.ends, self.Lambda_group, self.freq)
        else:
            return _fub2_jit(X, self.K, self.starts, self.ends, self.Lambda_group)

    def func_lb(self, X):
        return _flb_jit(X, self.Lambda_group)

    def dual(self, y):
        return _dual_jit(y, self.K, self.starts, self.ends, self.Lambda_group)


@jit(nopython=True)
def _fub1_jit(X, K, starts, ends, Lambda_group, freq):
    ub = 0.0
    for i in range(K):
        start, end = starts[i], ends[i]
        # decompose X_g into V_g
        Vg = X[start:end] / freq[start:end]
        ub += Lambda_group[i] * np.sqrt(np.sum(Vg * Vg))
    return ub


@jit(nopython=True)
def _fub2_jit(X, K, starts, ends, Lambda_group):
    ub = 0.0
    for i in range(K):
        start, end = starts[i], ends[i]
        # decompose X_g into V_g
        Vg = X[start:end]
        ub += Lambda_group[i] * np.sqrt(np.sum(Vg * Vg))
        X[start:end] = 0.0
    return ub


@jit(nopython=True)
def _flb_jit(X, Lambda_group):
    y = X / np.sqrt(np.sum(X * X))
    y = min(Lambda_group) * y
    lb = np.dot(X.T, y)[0][0]
    return lb


@jit(nopython=True)
def _dual_jit(y, K, starts, ends, Lambda_group):
    max_group_norm = 0.0
    for i in range(K):
        start, end = starts[i], ends[i]
        yg = y[start:end]
        temp = (np.sqrt(np.dot(yg * yg))[0][0]) / Lambda_group[i]
        max_group_norm = max(max_group_norm, temp)
    return max_group_norm

In [6]:
r = OGL1(Lambda=1, dim=p, starts=starts, ends=ends)
X = np.array([1.0, 4.2, 9.3, 12.6, 15.9, 8.4, 7.0]).reshape(-1,1)
print(r.func_ub(X,approx=1))
print(r.func_ub(X.copy(), approx=2))
print(r.func_lb(X))

61.79562331502589
85.26358866489885
56.40301410385796


In [7]:
r.Lambda_group

array([2.23606798, 2.23606798, 2.23606798])

In [15]:
p = int(1e4); ngrp=200; grp_size=200
generator = GenOverlapGroup(p, ngrp, grp_size)
starts, ends = generator.get_group()
r = OGL1(Lambda=1, dim=p, starts=starts, ends=ends)
np.random.seed(10)
X = np.random.randn(p,1)
print(r.func_ub(X, approx=1))
print(r.func_ub(X.copy(), approx=2))
print(r.func_lb(X))

9912.433804364251
19485.532211739825
1399.614867848165


In [ ]:
import numpy as np
from numpy.linalg import norm
a = np.array([2,3]); b = np.array([0,1])
print(norm(a) + norm(b))
a = np.array([2,0]); b = np.array([3,1])
print(norm(a) + norm(b))
a = np.array([2,3/2]); b = np.array([3/2,1])
print(norm(a) + norm(b))
print("===")
print(norm(np.array([2,3,1])))

In [25]:
import numpy as np
from numpy.linalg import norm
a = np.array([2,3]); b = np.array([0,1])
print(norm(a) + norm(b))
a = np.array([2,0]); b = np.array([3,1])
print(norm(a) + norm(b))
a = np.array([2,3/2]); b = np.array([3/2,1])
print(norm(a) + norm(b))
print("===")
print(norm(np.array([2,3,1])))

4.60555127546399
5.16227766016838
4.302775637731995
===
3.7416573867739413


In [13]:
from scipy.optimize import minimize_scalar
def f(x):
    return np.sqrt(4+x**2) + np.sqrt((3-x)**2 + 1)
res = minimize_scalar(f)
res.x

2.000000045899457

In [12]:
a = np.array([2,2]); b = np.array([1,1])
print(norm(a) + norm(b))

4.242640687119286


In [63]:
p = 7
starts, ends = [0, 3, 5], [2, 4, 6]
r = OGL1(Lambda=20, dim=p, starts=starts, ends=ends)
xk = 1.0* np.array([1, 2, 3, 4, 5, 6, 7]).reshape(-1, 1)
gradfxk = 0.1 * np.array([1, 2, 3, 4, 5, 6, 7]).reshape(-1, 1)
alphak = 0.2
uk = xk - alphak * gradfxk

proximal = np.zeros_like(xk)
for i in range(len(starts)):
    start, end = r.starts[i], r.ends[i]
    ukg = uk[start:end]
    ukg_norm = np.sqrt(np.dot(ukg.T, ukg))[0][0]
    if ukg_norm != 0:
        temp = 1 - ((r.Lambda_group[i] * alphak) / ukg_norm)
    else:
        temp = -1
    proximal[start:end] = max(temp, 0) * ukg
proximal.T

array([[0.        , 0.        , 0.        , 0.38619117, 0.48273896,
        2.19856805, 2.56499606]])

In [ ]:
0.517089950113724   1.034179900227449   1.551269850341173   3.036547791401228   3.795684739251535   4.959642013383156 5.786249015613682


In [50]:
temp = (proximal - uk) / alphak
dual_norm = r.dual(temp)
y = min(1, 1 / dual_norm) * temp
temp2 = proximal - uk
rproximal = 0
for i in range(3):
    start, end = r.starts[i], r.ends[i]
    proximalg = proximal[start:end]
    temp3 = np.sqrt(np.sum(proximalg*proximalg))
    rproximal += r.Lambda_group[i] * temp3
primal = np.dot(temp2.T, temp2)[0][0] / (2 * alphak) + rproximal
dual_negative = ((alphak / 2) * (np.dot(y.T, y)) + np.dot(uk.T, y))[0][0]
primal + dual_negative

0.0

In [52]:
temp = (proximal - uk) / alphak
dual_norm = r.dual(temp)
y = min(1, 1 / dual_norm) * temp
temp2 = proximal - uk
primal = np.dot(temp2.T, temp2)[0][0] / (2 * alphak) + r.func_ub(proximal)
dual_negative = ((alphak / 2) * (np.dot(y.T, y)) + np.dot(uk.T, y))[0][0]
primal + dual_negative

0.0

In [62]:
vk = np.zeros_like(uk)
for i in range(3):
    start, end = r.starts[i], r.ends[i]
    ukg = uk[start:end]
    ukg_norm = np.sqrt(np.sum(ukg*ukg))
    vk[start:end] = (r.Lambda_group[i]* alphak/ukg_norm) * ukg
(uk-vk).T

array([[0.51708995, 1.0341799 , 1.55126985, 3.03654779, 3.79568474,
        4.95964201, 5.78624902]])

In [60]:
vk.T

array([[2.31455025, 4.6291005 , 6.94365075, 4.41726104, 5.5215763 ,
        4.60178993, 5.36875492]])

In [61]:
(uk - proximal).T

array([[0.46291005, 0.9258201 , 1.38873015, 0.88345221, 1.10431526,
        0.92035799, 1.07375098]])

In [1]:
import numpy as np
from numpy.linalg import norm

In [20]:
def check_feasibility(z, starts, ends, bounds):
    for i in range(len(starts)):
        start, end = starts[i], ends[i]
        zg = z[start:end]
        zg_norm = np.sqrt(np.sum(zg*zg))
        if zg_norm > bounds[i]:
            z[start:end] = (bounds[i]/zg_norm) * zg
    return z


In [24]:
z = np.array([3.0,4.0,5.0])
starts = [0,1]
ends = [2,3]
check_feasibility(z, starts, ends, [5, 2])

array([3.        , 1.2493901 , 1.56173762])